# Imbalanced classification: credit card fraud detection

## Introduction

This example looks at the
[Kaggle Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud/)
dataset to demonstrate how
to train a classification model on data with highly imbalanced classes.

## First, vectorize the CSV data

In [ ]:
!wget https://www.dropbox.com/s/9vfy1vi6wsfkxyk/creditcard.csv.zip

--2021-03-17 17:39:01--  https://www.dropbox.com/s/9vfy1vi6wsfkxyk/creditcard.csv.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9vfy1vi6wsfkxyk/creditcard.csv.zip [following]
--2021-03-17 17:39:01--  https://www.dropbox.com/s/raw/9vfy1vi6wsfkxyk/creditcard.csv.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5cd9dfdc383877c28642aea094.dl.dropboxusercontent.com/cd/0/inline/BK1WhI7B_E9yzYB9FzOh1kfycpWZTIqTmi13QpgBahwPsBb1ObLHZs5M5HW325RTbDtfPipM9Sk-Qc6Td7HceYAdErbz0-aoRPawLCPbS-vJEhHA_ZszsnKIQ24ZMfsGS-8_7xfBj62CX5vHJJZauCq6/file# [following]
--2021-03-17 17:39:01--  https://uc5cd9dfdc383877c28642aea094.dl.dropboxusercontent.com/cd/0/inline/BK1WhI7B_E9yzYB9FzOh1kfycpWZTIqTmi13QpgBahwPsBb1ObLHZs5M5HW325RTbDtfPipM

In [ ]:
!unzip creditcard.csv.zip

Archive:  creditcard.csv.zip
  inflating: creditcard.csv          


In [ ]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)


HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


## Prepare a validation set

In [ ]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))


Number of training samples: 227846
Number of validation samples: 56961


## Analyze class imbalance in the targets

In [ ]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]


Number of positive samples in training data: 417 (0.18% of total)


## Normalize the data using training set statistics

In [ ]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std


## Build a binary classification model

In [ ]:
from tensorflow import keras

hid_size = 256
model = keras.Sequential(
    [
        keras.layers.Dense(
            hid_size, activation="relu", input_shape=(train_features.shape[-1],)
        ), # fully-connected y^1
        keras.layers.Dense(hid_size, activation="relu"), # y^2
        keras.layers.Dropout(0.3),
        keras.layers.Dense(hid_size, activation="relu"), # y^3
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"), # y^4
    ]
)
model.summary()

# x_1 = x_2 
# w_1 w_2
# w_1 = -w_2

# w_1 * x_1 + w_2 * x_2 = 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
__________________________________________________

## Train the model with `class_weight` argument

In [ ]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)


Epoch 1/30
112/112 - 8s - loss: 2.1683e-06 - fn: 41.0000 - fp: 26050.0000 - tn: 201379.0000 - tp: 376.0000 - precision: 0.0142 - recall: 0.9017 - val_loss: 0.1973 - val_fn: 5.0000 - val_fp: 3695.0000 - val_tn: 53191.0000 - val_tp: 70.0000 - val_precision: 0.0186 - val_recall: 0.9333
Epoch 2/30
112/112 - 6s - loss: 1.3584e-06 - fn: 32.0000 - fp: 8506.0000 - tn: 218923.0000 - tp: 385.0000 - precision: 0.0433 - recall: 0.9233 - val_loss: 0.0778 - val_fn: 9.0000 - val_fp: 1097.0000 - val_tn: 55789.0000 - val_tp: 66.0000 - val_precision: 0.0567 - val_recall: 0.8800
Epoch 3/30
112/112 - 6s - loss: 1.3031e-06 - fn: 32.0000 - fp: 8279.0000 - tn: 219150.0000 - tp: 385.0000 - precision: 0.0444 - recall: 0.9233 - val_loss: 0.0971 - val_fn: 9.0000 - val_fp: 1003.0000 - val_tn: 55883.0000 - val_tp: 66.0000 - val_precision: 0.0617 - val_recall: 0.8800
Epoch 4/30
112/112 - 6s - loss: 1.1633e-06 - fn: 29.0000 - fp: 8174.0000 - tn: 219255.0000 - tp: 388.0000 - precision: 0.0453 - recall: 0.9305 - val_l

In [ ]:
#    P(1)   N(0)
# T
# F

In [ ]:
# n_samples ~ 300000
# batch_size ~ 3000
# n_steps ~ 100
# n_epochs ~ 1

## Conclusions

At the end of training, out of 56,961 validation transactions, we are:

- Correctly identifying 66 of them as fraudulent
- Missing 9 fraudulent transactions
- At the cost of incorrectly flagging 441 legitimate transactions

In the real world, one would put an even higher weight on class 1,
so as to reflect that False Negatives are more costly than False Positives.

Next time your credit card gets  declined in an online purchase -- this is why.